## NB non fare girare per  L=2, 
**" The two-qubit optimization problem was shown to exhibit an additional symmetry-broken correlated phase"**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm,tnrange
from Qmodel import compute_H_and_LA
from SD import stochastic_descent,compute_fidelity
from random import gauss,seed
from functools import reduce
import os
import pandas as pd

In [2]:
def correlation(matrix,h):
    '''
    Given a matrix this function computes the correlation quantity Q(T) as in the paper.
    Matrix must have dimension (n_protocols,lenght_of_protocol)    
    '''
    n_row = matrix.shape[0]
    n_col = matrix.shape[1] 
    
    mean_hx = np.array([matrix[:,i].mean() for i in range(n_col)]) #mean over all protocols at fixed time
    avg_over_h = np.array([np.array([ (matrix[i,j]-mean_hx[j])**2 for i in range(n_row)]).sum()/n_row for j in range(n_col)])
    avg_over_Nt = (1/((h*h)*n_col))*(avg_over_h.sum())
    return  avg_over_Nt

In [3]:
def coefficients_wave_functions(L):
    coefficients = np.array([gauss(0.0, 1.0) for _ in range(2**(L+1))])
    norm = np.sqrt(reduce(lambda sum,x: sum + x*x, coefficients, 0.0))
    real = np.split(coefficients, 2)[0]
    imag = np.split(coefficients, 2)[1]
    return np.array([real / norm + 1.j*(imag / norm) for real,imag in zip(real,imag)],dtype=np.complex128)

In [4]:
#Starting qstates and their normalization
qstart = np.array([-1/2 - (np.sqrt(5))/2 ,1], dtype=complex)
qtarget = np.array([+1/2 + (np.sqrt(5))/2 ,1], dtype=complex)
qstart=qstart/np.sqrt(np.vdot(qstart,qstart))
qtarget=qtarget/np.sqrt(np.vdot(qtarget,qtarget))
print("qstart",qstart)

qstart [-0.85065081+0.j  0.52573111+0.j]


In [5]:
#SD parameters
L = 3
h = 4
h_list = [-h,h]
nsteps = 100
exp_decay = False
metropolis = False


qstart = coefficients_wave_functions(L=L) # Start from a random 2^L state 
qtarget = coefficients_wave_functions(L=L)  # Remember to change first component when things will work.
print("Initial Fidelity :",compute_fidelity(qstart,qtarget))

Initial Fidelity : 0.3151694102501727


In [8]:
compute_H_and_LA(L=3,g=1,field=4)

{'H': array([[-2.25, -2.  , -2.  , -0.  , -2.  , -0.  , -0.  , -0.  ],
        [-2.  , -0.25, -0.  , -2.  , -0.  , -2.  , -0.  , -0.  ],
        [-2.  , -0.  , -0.25, -2.  , -0.  , -0.  , -2.  , -0.  ],
        [-0.  , -2.  , -2.  ,  0.75, -0.  , -0.  , -0.  , -2.  ],
        [-2.  , -0.  , -0.  , -0.  , -0.25, -2.  , -2.  , -0.  ],
        [-0.  , -2.  , -0.  , -0.  , -2.  ,  0.75, -0.  , -2.  ],
        [-0.  , -0.  , -2.  , -0.  , -2.  , -0.  ,  0.75, -2.  ],
        [-0.  , -0.  , -0.  , -2.  , -0.  , -2.  , -2.  ,  0.75]]),
 'eigval': array([ 6.16876758, -6.26099189,  1.61785763, -2.52563332, -1.81155281,
         2.31155281,  2.31155281, -1.81155281]),
 'eigvect': array([[-2.24325662e-01,  5.51237364e-01,  4.84625871e-01,
          6.41056262e-01, -2.39507244e-16,  1.12612540e-16,
         -1.24482371e-17, -1.80865671e-16],
        [ 3.14757602e-01,  3.68501433e-01, -3.12410645e-01,
          2.94494110e-02, -4.56341942e-01, -3.59489460e-01,
          4.39742179e-01, -5.92515743e

In [9]:
print("-------------PARAMETERS for SD---------------")
print("Number of qubits (L):", L)
print("Magnetic fields(h):", h)
print("Timesteps (n_steps):", nsteps)
print("\n")
print("\n")

#parameters for Fig. pag 2 ---- Calculation of fidelity and Q(T)
#step_in_time_grid = 0.1
times_first_part=np.arange(0,1,0.05)
times_second_part=np.arange(1,4.1,0.1)
times=np.concatenate([times_first_part,times_second_part])
#times = np.arange(0,4,0.1)#+step_in_time_grid,step=step_in_time_grid)
iter_for_each_time = 10

fidelity_for_txt = []
print("--------------PARAMETERS for Plotting-------------")
print("Timegrid:", times)
print("Repetition at each timestep:", iter_for_each_time)
print("\n")

params_dict = {"L":L, "h":h, "timesteps":nsteps, "exp_decay":exp_decay, "metropolis":metropolis, "times":times, "iter_for_each_time": iter_for_each_time}
params_df = pd.DataFrame.from_dict(params_dict, orient="index")

-------------PARAMETERS for SD---------------
Number of qubits (L): 3
Magnetic fields(h): 4
Timesteps (n_steps): 100




--------------PARAMETERS for Plotting-------------
Timegrid: [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7
 1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7  2.8  2.9  3.   3.1
 3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.  ]
Repetition at each timestep: 10




In [10]:
#UNCOMMENT TO RUN
print("WARNING! Attenzione a non sovrascrivere")

#save run parameters and date in custom named folder
#os.chdir("C:\Users\GUIDA\Desktop\CODE_MONTANGERO\SD_WITH_REPLACEMENT")
custom_name_dir = "final_data_fixed_nsteps"
os.system("mkdir "+custom_name_dir)
os.system("mkdir "+custom_name_dir+"/protocols")
params_df.to_csv(custom_name_dir+"/parameters.csv")

intermediete_result = True

for T in tqdm(times):
    temp_fid = []
    best_prot = []
    print("Running time:", T)
    for _ in range(iter_for_each_time):

        best_protocol, fidelity = stochastic_descent(qstart, qtarget, L, T, nsteps, 
                        field_list = h_list, verbose = True, check_norm = True)

        temp_fid.append(fidelity[-1]) #at fixed T we will have "iter_for_each_time" evaluations of fidelity
        best_prot.append(best_protocol) #all iter_for_each_time best protocols are stored in this variable 
                                        #and saved when new rounf over T starts
    fidelity_for_txt.append(temp_fid) #fidelity evaluations are stored in the same "fidelity_fot_txt"
                                      #variable that will have dimension len(times)*iter_for_each_time
    best_prot = np.array([best_prot])
    with open(custom_name_dir +'/protocols/testT'+str(round(T, 2))+'.npy', 'wb') as f:
        np.save(f,best_prot)
    f.close()
    
    if intermediete_result and T !=0: #if T = 0 q cannot be computed
        data = np.load(custom_name_dir +'/protocols/testT'+str(round(T, 3))+'.npy')[0,:,:] #first dimension is redundant 
        print("Mean fidelity:", np.array(temp_fid).mean())
        print("Q value is:", correlation(data, h))
        print("\n")
        
#fidelity values are saved at the end
np.savetxt(custom_name_dir + '/fidelity_SD.txt', fidelity_for_txt, delimiter = ',',header="Matrix with as entries the values of fidelity dimension times x iterations")


  0%|                                                                                           | 0/51 [00:00<?, ?it/s]

WARNING! Attenzione a non sovrascrivere
Running time: 0.0
Warning ---> Norm is not conserved
5.751979700042875e+48
Warning ---> Norm is not conserved
5.97172516462132e+48
Warning ---> Norm is not conserved
4.8742758244585275e+48
Warning ---> Norm is not conserved
4.4935985568605326e+48
Warning ---> Norm is not conserved
4.864780841569038e+48
Warning ---> Norm is not conserved
4.556527260824214e+48
Warning ---> Norm is not conserved
4.864780841569038e+48
Warning ---> Norm is not conserved
5.7465519106307984e+48
Warning ---> Norm is not conserved
4.6188781191529155e+48
Warning ---> Norm is not conserved
6.664061143459162e+48
Warning ---> Norm is not conserved
4.480213376353791e+48
Warning ---> Norm is not conserved
4.7628499235873277e+48
Warning ---> Norm is not conserved
5.952255566213007e+48
Warning ---> Norm is not conserved
5.770547314100427e+48
Warning ---> Norm is not conserved
5.97172516462132e+48
Warning ---> Norm is not conserved
5.738512448978618e+48
Warning ---> Norm is not co

Warning ---> Norm is not conserved
4.2812224418107694e+48
Warning ---> Norm is not conserved
4.227585255824589e+48
Warning ---> Norm is not conserved
6.184570463193954e+48
Warning ---> Norm is not conserved
4.083137714831412e+48
Warning ---> Norm is not conserved
4.4935985568605326e+48
Warning ---> Norm is not conserved
4.5121597553391336e+48
Warning ---> Norm is not conserved
4.307259105417862e+48
Warning ---> Norm is not conserved
5.874957411867324e+48
Warning ---> Norm is not conserved
6.664061143459162e+48
Warning ---> Norm is not conserved
4.288204825048403e+48
Warning ---> Norm is not conserved
5.7902642802360875e+48
Warning ---> Norm is not conserved
6.150731270645609e+48
Warning ---> Norm is not conserved
4.3558082210206746e+48
Warning ---> Norm is not conserved
5.97172516462132e+48
Warning ---> Norm is not conserved
4.132078540825697e+48
Warning ---> Norm is not conserved
5.874957411867324e+48
Warning ---> Norm is not conserved
4.6188781191529155e+48
Warning ---> Norm is not c

Warning ---> Norm is not conserved
4.853269603730269e+48
Warning ---> Norm is not conserved
5.774088484957061e+48
Warning ---> Norm is not conserved
5.770547314100427e+48
Warning ---> Norm is not conserved
6.001003127074345e+48
Warning ---> Norm is not conserved
6.957998909269985e+48
Warning ---> Norm is not conserved
6.07143716449526e+48
Warning ---> Norm is not conserved
5.764258776965527e+48
Warning ---> Norm is not conserved
4.4935985568605326e+48
Warning ---> Norm is not conserved
4.3539608018338745e+48
Warning ---> Norm is not conserved
6.677483009892705e+48
Warning ---> Norm is not conserved
4.3653609256721136e+48
Warning ---> Norm is not conserved
4.3653609256721136e+48
Warning ---> Norm is not conserved
6.07143716449526e+48
Warning ---> Norm is not conserved
6.184570463193954e+48
Warning ---> Norm is not conserved
5.774088484957061e+48
Warning ---> Norm is not conserved
6.664061143459162e+48
Warning ---> Norm is not conserved
5.994632732080118e+48
Warning ---> Norm is not cons

  0%|                                                                                           | 0/51 [00:04<?, ?it/s]


6.803006526500539e+48
Warning ---> Norm is not conserved
4.307259105417862e+48


KeyboardInterrupt: 

In [ ]:
#PLOT 
#os.chdir("C:\Users\GUIDA\Desktop\CODE_MONTANGERO\SD_WITH_REPLACEMENT")
q=[]
for T in times[1:]:
    data = np.load(custom_name_dir +"/protocols/testT"+str(round(T, 2))+".npy")[0,:,:] #first dimension is redundant 
    #!warning: correlation normalization term musb be changed according to field values
    q.append(correlation(data,h))
    
loaded_fidelity = pd.read_csv(custom_name_dir +'/fidelity_SD.txt', skiprows=1,header=None)

q[0]=0
fig, ax = plt.subplots(figsize=(10,7))
#plot Fidelity values
ax.errorbar(times,loaded_fidelity.mean(axis=1).values, yerr=loaded_fidelity.std(axis=1).values, color="r")
ax.scatter(times,loaded_fidelity.mean(axis=1).values,color='r',label="Fidelity")
#plot Q values
ax.plot(times[1:], q, marker="o", color="goldenrod", markersize=7, label="q(T)")
ax.vlines(0.5,-0.05,1.05, color="b", linestyle="-.")
ax.vlines(2.4,-0.05,1.05, color="b", linestyle="-.")
ax.set_ylim(-0.05,1.05)
ax.set_title(r"Phase Diagram"+"\n"+"Iterations for Each Point = "+str(iter_for_each_time), fontsize=18)
ax.set_xlabel("T [a.u.]", fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.legend(fontsize=13, loc=5)
ax.grid()
fig.tight_layout()
fig.savefig("phase_diagram_final.pdf")
plt.show()